In [76]:
from keras.models import Sequential
import time
from keras.layers import Input, LSTM, RepeatVector
from keras.models import Model
from utils import import_seq_data
import glob
import numpy as np
from midi_parser import *
NOTE_SIZE = 156

In [ ]:
start = time.time()
X,y = import_seq_data() # need a MUCH faster way to write this function. Need to save them somewhere, so it won't take 
end = time.time()
print end - start
# as much time to process. definitely could use numpy. will rewrite the function after. Doshi is buns at programing lol

In [28]:
np.shape(X[0]) #ok, so the Xs and ys have the same size. 
np.shape(y[0]) # hngi? <-- need to ensure X and Y always have same size? I feel liek this is already a thing. 
# can add this functionality if needed. doshis thing has it but it is way too slow. 

(7724, 156)

In [61]:
# split out a validation set
split_at = len(X) - len(X) / 10
X_train = X[:split_at]
X_val = X[split_at:]
y_train = y[:split_at]
y_val = y[split_at:]

In [ ]:
NUM_NOTES = 156
HIDDEN_SIZE = 500
MAXLEN = len(X[0]) # the maximum length of the training sample.
LAYERS = 1

In [68]:
model = Sequential()
model.add(LSTM(HIDDEN_SIZE, input_shape=(MAXLEN, NUM_NOTES))) # variable size
model.add(RepeatVector(MAXLEN)) # input to decoder = hidden size thing repeated for each time step


for _ in range(LAYERS):
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))
    
model.add(TimeDistributed(Dense(len(chars))))
model.add(Activation('softmax'))


In [58]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

247